In [ ]:
import numpy as np

20天的數據總共有 (4320*24) 的資料，有18個資料特徵，將原始資料以月份先做劃分，因為取20天的資料每天24小時，所以會有一個 18(features)*20(days)*24(hours) 的資料，透過這樣的預處理就可以把資料洗成橫向且連續的資料。

In [ ]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day*24:(day+1)*24] = raw_data[18 *
                                                (20*month+day):18*(20*month+day+1):]
    month_data[month] = sample

依據實驗需求需要使用9小時的資料去預測第十個小時的PM2.5，剛剛分割過後每一個月都會有480小時，但"每個月的前 9 小時不能形成訓練樣本，因為它們沒有足夠的數據來預測第 10 小時。因此，從總小時數 480 中減去 9，我們得到 471 個有效的訓練樣本。”

In [ ]:
x = np. empty([12 * 471, 18 * 9], dtype=float)
y = np. empty([12 * 471, 1], dtype=float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,
                                                                    day * 24 + hour: day * 24 + hour + 9].reshape(1, -1)
            y[month * 471 + day * 24 + hour,
                0] = month_data[month][9, day * 24 + hour + 9]
print(x)
print(y)

計算平均值和標準差

In [ ]:
mean_x = np.mean(x, axis=0)  # 每個特徵的平均值
std_x = np.std(x, axis=0)    # 每個特徵的標準差

使用 NumPy 的向量化操作可以一次處理整個陣列，而不需要單獨處理每個元素。

In [ ]:
x_normalized = (x - mean_x) / std_x
std_x[std_x == 0] = 1

使用 numpy 進行分割

In [ ]:
train_size = int(len(x) * 0.8)
x_train_set, x_validation = x[:train_size], x[train_size:]
y_train_set, y_validation = y[:train_size], y[train_size:]

檢查數據筆數

In [ ]:
print(len(x_train_set))
print(len(x_validation))
print(len(y_train_set))
print(len(y_validation))

權重初始化

In [ ]:
dim = 18 * 9 + 1
w = np.zeros([dim, 1])

偏置項（bias term）是一個常數值（通常表示為 b 或 w0），它與模型中的特徵值（x1, x2, x3, ...）無關。偏置項允許模型的輸出偏離原點，即使所有特徵值都為零時也可以有一個非零的輸出。像是:

$$
y=w0+w1x1+w2x2+...+wnxn

$$

其中 y 是預測值，w0 是偏置項，w1 是特徵權重，x1 是特徵值。

In [ ]:
x = np.hstack([np.ones((x.shape[0], 1)), x])

初始化變量

In [ ]:
learning_rate = 1
iter_time = 1200
adagrad = np.zeros([dim, 1])
eps = 0.0000000001

訓練迴圈

In [ ]:
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)  # RMSE
    if (t % 100 == 0):
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y)  # 梯度
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)

讀取測試數據

In [ ]:
testdata = pd.read_csv('./test.csv', header=None, encoding='big5')
test_data = testdata.iloc[:, 2:]
test_data[test_data == 'NR'] = 0
test_data = test_data.to_numpy()

正規化測試數據

In [ ]:
test_x = np.empty([240, 18*9], dtype=float)
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18 * (i + 1), :].reshape(1, -1)

添加篇置項

In [ ]:
test_x = np.concatenate((np.ones([240, 1]), test_x), axis=1).astype(float)

加載訓練後的權重

In [ ]:
w = np.load('./weight.npy')